In [1]:
import os.path
import tensorflow as tf
import warnings
import numpy
import scipy
import helper
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    image_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)
print('load_vgg passed')

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    init = tf.truncated_normal_initializer(stddev = 0.01)
    regu = tf.contrib.layers.l2_regularizer(1e-3)
    def conv_1x1(x, num_classes, init = init):
        return tf.layers.conv2d(x, num_classes, 1, padding = 'same', kernel_initializer = init, kernel_regularizer = regu)

    def upsample(x, num_classes, depth, strides, init = init):
        return tf.layers.conv2d_transpose(x, num_classes, depth, strides, padding = 'same', kernel_initializer = init, kernel_regularizer = regu)
    
    layer_7_1x1 = conv_1x1(vgg_layer7_out, num_classes)
    layer_4_1x1 = conv_1x1(vgg_layer4_out, num_classes)
    layer_3_1x1 = conv_1x1(vgg_layer3_out, num_classes)

    #implement the first transposed convolution layer
    upsample1 = upsample(layer_7_1x1, num_classes, 5, 2)
    layer1 = tf.layers.batch_normalization(upsample1)
    #add the first skip connection from the layer_4_1x1    
    layer1 = tf.add(layer1, layer_4_1x1)

    #implement the another transposed convolution layer
    upsample2 = upsample(layer1, num_classes, 5, 2)
    layer2 = tf.layers.batch_normalization(upsample2)
    #add the second skip connection from the layer_3_1x1        
    layer2 = tf.add(layer2, layer_3_1x1)

    return upsample(layer2, num_classes, 14, 8)    

tests.test_layers(layers)
print("test_layers passed")

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1,num_classes))
    # define loss function
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label))
    # define training operation
    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss
tests.test_optimize(optimize)
print('test optimization passed')

def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate, patience = 50):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    
    low_mark = 9999.0
    counter =0
    
    checkpointFile = os.path.join('./ckpt', 'vgg16FNN.ckpt')
    sess.run(tf.global_variables_initializer())
        
    print("Training...")
    print()
    for epoch in range(epochs):
        print("EPOCH {} ...".format(epoch+1))
        
        kr = 0.6
        initial_lr = 0.00005
        
        if epoch > 5 and epoch <= 15:
            lr = initial_lr / 7
        elif epoch > 15 and epoch <= 25:
            lr = initial_lr / 50
        elif epoch > 25:
            lr = initial_lr / 350
        else:
            lr = initial_lr

        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: kr, learning_rate: initial_lr})
            counter += 1
            if loss < (low_mark - .001):
                tf.train.Saver().save(sess, checkpointFile)
                print("low mark")
                low_mark = loss
                counter = 0
            if counter >= patience:
                print("early stop!")
                break
            print("Loss: = {:.3f} - counter: {c}/{p} ".format(loss, c=counter, p=patience))
            
        print()
#tests.test_train_nn(train_nn)
#print('test_train_nn passed')

def run():
    num_classes = 2
    epochs = 80
    batch_size = 5
    patience = 50
    
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    checkpointFile = os.path.join('./ckpt', 'vgg16FNN.ckpt')
    
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    #  no way!
    
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        vgg_input, keep_prob, vgg_layer3, vgg_layer4, vgg_layer7 = load_vgg(
            sess, vgg_path)

        nn_last_layer = layers(vgg_layer3, vgg_layer4, vgg_layer7, num_classes)

        correct_label = tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], num_classes])
        learning_rate = tf.placeholder(tf.float32)

        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)
        sess.run(tf.global_variables_initializer())        

        # Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, vgg_input,
             correct_label, keep_prob, learning_rate, patience)

        #restore the best model
        tf.train.Saver().restore(sess, checkpointFile)
        
        # Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, vgg_input)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()


TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0
Tests Passed
load_vgg passed
Tests Passed
test_layers passed
Tests Passed
test optimization passed
Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Training...

EPOCH 1 ...
low mark
Loss: = 1.564 - counter: 0/50 
low mark
Loss: = 1.129 - counter: 0/50 
low mark
Loss: = 1.052 - counter: 0/50 
low mark
Loss: = 0.988 - counter: 0/50 
low mark
Loss: = 0.925 - counter: 0/50 
Loss: = 1.028 - counter: 1/50 
Loss: = 0.977 - counter: 2/50 
low mark
Loss: = 0.888 - counter: 0/50 
low mark
Loss: = 0.834 - counter: 0/50 
low mark
Loss: = 0.791 - counter: 0/50 
Loss: = 0.811 - counter: 1/50 
Loss: = 0.794 - counter: 2/50 
low mark
Loss: = 0.786 - counter: 0/50 
low mark
Loss: = 0.764 - counter: 0/50 
Loss: = 0.804 - counter: 1/50 
Loss: = 0.773 - counter: 2/50 
low mark
Loss: = 0.750 - counter: 0/50 
low mark
Loss: = 0.746 - counter: 0/50 
low mark
Loss: = 0.722 - counter: 0/50 
low mark
Loss: = 0.718 - cou

early stop!

EPOCH 33 ...
early stop!

EPOCH 34 ...
early stop!

EPOCH 35 ...
early stop!

EPOCH 36 ...
early stop!

EPOCH 37 ...
early stop!

EPOCH 38 ...
early stop!

EPOCH 39 ...
low mark
Loss: = 0.085 - counter: 0/50 
Loss: = 0.092 - counter: 1/50 
Loss: = 0.106 - counter: 2/50 
Loss: = 0.138 - counter: 3/50 
Loss: = 0.132 - counter: 4/50 
Loss: = 0.116 - counter: 5/50 
Loss: = 0.118 - counter: 6/50 
Loss: = 0.091 - counter: 7/50 
Loss: = 0.139 - counter: 8/50 
Loss: = 0.158 - counter: 9/50 
Loss: = 0.104 - counter: 10/50 
Loss: = 0.147 - counter: 11/50 
Loss: = 0.151 - counter: 12/50 
Loss: = 0.179 - counter: 13/50 
Loss: = 0.107 - counter: 14/50 
Loss: = 0.103 - counter: 15/50 
Loss: = 0.130 - counter: 16/50 
Loss: = 0.100 - counter: 17/50 
Loss: = 0.123 - counter: 18/50 
Loss: = 0.133 - counter: 19/50 
Loss: = 0.124 - counter: 20/50 
low mark
Loss: = 0.076 - counter: 0/50 
Loss: = 0.075 - counter: 1/50 
Loss: = 0.144 - counter: 2/50 
Loss: = 0.136 - counter: 3/50 
Loss: = 0.109 